## Despite the title, 
# Run this after doing the validation step

In [ ]:
import os
import dask.array as da
import re
from aicsimageio import AICSImage
import numpy as np
import datetime

### Fill in the base path. 
#### Output files will have names dictated by date at runtime

In [ ]:
base_path = "/vast/imaging/Sabrina/New_new/1064/"

today = datetime.datetime.today().strftime("%Y%m%d")

dataset_label = base_path.split('/')[-2]

original_met_path = os.path.join(base_path,'output_mets')
met_path = os.path.join(original_met_path,'CORRECTED_METS')

preValidated_filename = "%s_preValidated_%s_results.csv" % (today,dataset_label)
validated_filename = "%s_validated_%s_results.csv" % (today,dataset_label)

print(preValidated_filename)
print(validated_filename)




#### Get all the required files

In [ ]:
met_path_files = os.listdir(met_path)
original_met_path_files = os.listdir(original_met_path)
mets = [m for m in met_path_files if m.endswith('tif')]
results = [r for r in original_met_path_files if r.endswith('csv')]
notes_files = [n for n in met_path_files if n.startswith('notes')]



#### Process Files

In [ ]:
notes = []
for notes_file in notes_files:
    with open(os.path.join(met_path,notes_file),'r') as f:
        f.readline()
        for line in f:
            line = line.strip()
            l = line.split(',')
            x = []
            for idx,res in enumerate(l):
                if idx==0:
                    x.append(int(res))
                else:
                    try:
                        x.append(float(res))
                    except:
                        x.append(res)
            notes.append(x)

In [ ]:
original_results = []
for results_file in results:
    with open(os.path.join(original_met_path,results_file),'r') as f:
        f.readline()
        for line in f:
            line = line.strip()
            l = line.split(',')
            x = []
            for idx,res in enumerate(l):
                if idx==0:
                    x.append(int(res))
                else:
                    try:
                        x.append(float(res))
                    except:
                        x.append(res)
            original_results.append(x)

            

In [ ]:
pre_validated_results = []

for r in original_results:
   # print(r)
    r_new = r
    try:
        associated_note = [n for n in notes if n[0]==r[0]][0]
        associated_note = associated_note[-1]
    except IndexError:
        associated_note = 'No note found'
    #print(associated_note[-1])
    r_new.append(associated_note)
    res_string = ','.join(str(elem) for elem in r_new)
    pre_validated_results.append(res_string)
    


In [ ]:
new_res_file = os.path.join(base_path,preValidated_filename)
with open(new_res_file,'w') as f:
    headings = 'MET_ID,Volume,001,010,011,100,101,110,111,min_dist,max_dist,mean_dist,Nearest Vessel Thickness,notes'
    print(headings,file=f)
    for r in pre_validated_results:       
        print(r,file=f)
    
    

print("Results are to be found in %s" % new_res_file)

## Recalculate all the met volumes
#### This may take a while and it will print a bunch of stuff

In [ ]:
post_validated_results = []
idx=0
for r in pre_validated_results:
    print(r)
    r_new = r.split(',')
    met_num = int(r_new[0])
    metFile = os.path.join(met_path,'met_%i_mask.tif' % met_num)
    if os.path.exists(metFile):
        print(idx)
        idx+=1
        print('%i - File Found' % met_num)
        
        maskImg = AICSImage(metFile).dask_data
        scores = np.zeros(7)
        
        for c in range(0,7):
            try:
                thisChannel = maskImg[0,c,:,:,:]
                n_non_zero = np.count_nonzero(thisChannel).compute()
                scores[c] = n_non_zero
                scores_normalised = [x / np.sum(scores) for x in scores]
            except:
                scores_normalised = np.zeros(7)
        this_met_results = [met_num,r_new[1]]
        for s in scores_normalised:
            this_met_results.append(s)
        this_met_results.extend(r_new[9:14])
        print(this_met_results)
    else:
        print('File Not Found')

    post_validated_results.append(this_met_results)

### Write out validated results

In [ ]:
print(validated_filename)
output_path = os.path.join(base_path, validated_filename)

with open(output_path,'w') as f:
    for l in post_validated_results:
        res_string = ','.join(str(elem) for elem in l)
        print(res_string,file=f)

print("File can be found in %s" % output_path)
